In [8]:
import { ChromaClient } from "chromadb";
import fs from 'fs';
import csv from 'csv-parser';
import path from 'path';
const client:ChromaClient  = new ChromaClient({ path: "http://localhost:8000" });


In [10]:
type Data = {
    word: string
    definition: string
    sources?: string
  }
  
  type GlossaryProps = {
    data: Data[];
  }

In [4]:
const collection = await client.getOrCreateCollection({
    name: "glosario",
    model_validate: false,
});

3:5 - Argument of type '{ name: string; model_validate: boolean; }' is not assignable to parameter of type 'CreateCollectionParams'.
3:5 - Object literal may only specify known properties, and 'model_validate' does not exist in type 'CreateCollectionParams'.


In [18]:
async function glosario() {
    const results: Data[] = [];
    const filePath = path.join(process.cwd(), 'db', 'glosario.csv');
  
    await new Promise((resolve, reject) => {
      fs.createReadStream(filePath)
        .pipe(csv({ separator: ',', escape: '"', headers: ['word', 'definition', 'sources'] }))
        .on('data', (data) => {
          results.push({
            word: data.word, definition: data.definition, sources: data.sources || ''
          });
        })
        .on('end', () => {
          resolve(results);
        })
        .on('error', (error: any) => {
          reject(error);
        });
    });
  
    return results;
  }

In [19]:
const glos = await glosario();


In [21]:
glos[0]

{
  word: 'Antifrágil',
  definition: 'Habilidad de un sistema para fortalecerse y evolucionar debido al estrés, la volatilidad y los retos enfrentados.',
  sources: ''
}


In [ ]:
await collection.add({
    documents: [
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids: ["id1", "id2"],
});